# Creating a LlamaIndex RAG Pipeline with NL2SQL and Metadata Filtering!

## Install Dependencies

In [2]:
!pip install -qU llama-index openai anthropic wikipedia llama-index-readers-wikipedia sqlalchemy pandas nest-asyncio anthropic==0.28.1

In [3]:
!pip install -qU wandb llama-index-callbacks-wandb grpcio-tools

In [4]:
!pip install -qU llama-index-vector-stores-qdrant qdrant-client --no-deps

In [6]:
import nest_asyncio

nest_asyncio.apply()

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Task 2: Set Env Variables and Set Up WandB Callback

In [7]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

In [8]:
import os
import getpass

os.environ["WANDB_API_KEY"] = getpass.getpass("WandB API Key: ")

In [84]:
import llama_index
from llama_index.core import set_global_handler

set_global_handler("wandb", run_args={"project": "llama-index-rag-v1"})
wandb_callback = llama_index.core.global_handler

# OPTIONAL ADVANCED PATH:

In [5]:
!pip install -qU llama-index-llms-anthropic

In [10]:
os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key: ")


In [42]:
# from llama_index.llms.anthropic import Anthropic
# from llama_index.core import Settings

# Settings.llm = Anthropic(model="claude-3-5-sonnet-20240620")


from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

Settings.llm = OpenAI(model="gpt-4o")

# Task 3: Settings

In [44]:
from llama_index.embeddings.openai import OpenAIEmbedding
Settings.embedding = OpenAIEmbedding(model="text-embedding-3-small")

# Task 4: Index Creation

## Data Collection

In [14]:
from llama_index.readers.wikipedia import WikipediaReader

movie_list = [
    "Dune (2021 film)",
    "Dune: Part Two",
    "The Lord of the Rings: The Fellowship of the Ring",
    "The Lord of the Rings: The Two Towers",
]

wiki_docs = WikipediaReader().load_data(pages=movie_list, auto_suggest=False)

In [ ]:
wiki_docs

# Initializing our VectorStoreIndex with QDrant


In [45]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient, models

client = QdrantClient(location=":memory:")

client.create_collection(
    collection_name="movie_wikis",
    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE)
)

True

In [47]:
from llama_index.core import VectorStoreIndex
from llama_index.core import StorageContext

vector_store = QdrantVectorStore(client=client, collection_name="movie_wikis")

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    [],
    storage_context=storage_context,
)

wandb: Logged trace tree to W&B.


## Node Construction

In [48]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter
from llama_index.core.extractors import TitleExtractor

pipeline = IngestionPipeline(transformations=[TokenTextSplitter()])

for movie, wiki_doc in zip(movie_list, wiki_docs):
  nodes = pipeline.run(documents=[wiki_doc])
  for node in nodes:
      node.metadata = {"title" : movie}
  index.insert_nodes(nodes)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


## ❓ Question #1:
What metadata fields will the nodes in our index have?

Seems like metadata field only has title so far.

In [36]:
for node in index.vector_store.get_nodes():
    print(node.metadata)

{'title': 'The Lord of the Rings: The Fellowship of the Ring'}
{'title': 'The Lord of the Rings: The Two Towers'}
{'title': 'The Lord of the Rings: The Fellowship of the Ring'}
{'title': 'Dune: Part Two'}
{'title': 'Dune: Part Two'}
{'title': 'The Lord of the Rings: The Two Towers'}
{'title': 'The Lord of the Rings: The Fellowship of the Ring'}
{'title': 'Dune (2021 film)'}
{'title': 'Dune (2021 film)'}
{'title': 'The Lord of the Rings: The Fellowship of the Ring'}
{'title': 'The Lord of the Rings: The Two Towers'}
{'title': 'Dune (2021 film)'}
{'title': 'The Lord of the Rings: The Two Towers'}
{'title': 'Dune (2021 film)'}
{'title': 'The Lord of the Rings: The Fellowship of the Ring'}
{'title': 'Dune (2021 film)'}
{'title': 'Dune (2021 film)'}
{'title': 'Dune: Part Two'}
{'title': 'Dune: Part Two'}
{'title': 'The Lord of the Rings: The Fellowship of the Ring'}
{'title': 'Dune (2021 film)'}
{'title': 'Dune (2021 film)'}
{'title': 'Dune (2021 film)'}
{'title': 'Dune: Part Two'}
{'title'

## Persisting and Loading Stored Index with Weights and Biases

In [37]:
wandb_callback.persist_index(index, index_name="movie-index-qdrant")

wandb: Adding directory to artifact (/Users/waseem/code/AIM3/aim_exercises/exercise_9/wandb/run-20240709_202615-3k0iw9fg/files/storage)... Done. 2.0s


In [49]:
from llama_index.core import load_index_from_storage

storage_context = wandb_callback.load_storage_context(
    artifact_url="test11111111111/llama-index-rag-v1/movie-index-qdrant:v0"
)

wandb:   4 of 4 files downloaded.  


## ❓ Question #2:
Provide a screenshot of your index version history as shown in WandB.
![alt text](./index-versions.png)


# Task 5: Simple RAG - QueryEngine


In [50]:
simple_rag = index.as_query_engine()


In [51]:
for k, v in simple_rag.get_prompts().items():
  print(v.get_template())
  print("\n~~~~~~~~~~~~~~~~~~\n")

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 

~~~~~~~~~~~~~~~~~~

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Refined Answer: 

~~~~~~~~~~~~~~~~~~



In [52]:
response = simple_rag.query("Who is the evil Wizard in the story?")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


In [53]:
response


Response(response='The evil wizard in the story is Saruman the White.', source_nodes=[NodeWithScore(node=TextNode(id_='01f6cf84-7244-4a2a-aceb-618e06a8b58a', embedding=None, metadata={'title': 'The Lord of the Rings: The Two Towers'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='173944', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='ac6f3e4c23f2135576d6801137796a752b31662fe2c6066d3bcbb22349c86214'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='2a747d71-230e-45dd-80ce-ce2da41a1a79', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='82d39dabbceda501ed8c228516bd575a19a2cfccdb3d2d96614393016775f698'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='784156af-e0f4-4ed0-b8b8-3b1876766b18', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2c2b024ccb5d14863fb3b4b390ace7790ed4b2f8200c09d2c7e48fa25da87de5')}, text='the entire battle in the book.\nOn the way to Helm\'s 

In [54]:
response = simple_rag.query("Who are the giant beings that roam across the world?")


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


In [55]:
response

Response(response='The giant beings that roam across the world are the sandworms.', source_nodes=[NodeWithScore(node=TextNode(id_='158a9252-f43f-43c2-9527-930971cd0f0e', embedding=None, metadata={'title': 'Dune (2021 film)'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='52659577', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='5d384c74de181e443fc55476b3c856e0cd96b4b9648056e317e1d5d80fae66a7'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='b0a69335-70db-4303-a500-e9e1200487e9', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='34f89954ccdaab1dece6896a48e7cdfe96729b6867cafd4aab92c33563aadcff'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='3894509a-f73e-4ea1-a53c-2e99bf288f10', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='2e5fdf608b028722584169e175d946cd3a6cdde3bda2109c5f0fb2089ec3da9c')}, text='of Mesopotamian ziggurat architecture, Egyptian references, W

In [56]:
print([x.metadata["title"] for x in response.source_nodes])


['Dune (2021 film)', 'The Lord of the Rings: The Fellowship of the Ring']


# Task 6: Auto Retriever Functional Tool


In [57]:
from llama_index.core.tools import FunctionTool
from llama_index.core.vector_stores.types import (
    VectorStoreInfo,
    MetadataInfo,
    ExactMatchFilter,
    MetadataFilters,
)
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

from typing import List, Tuple, Any
from pydantic import BaseModel, Field

top_k = 3

vector_store_info = VectorStoreInfo(
    content_info="semantic information about movies",
    metadata_info=[MetadataInfo(
        name="title",
        type="str",
        description='title of the movie, one of ["Dune (2021 film)", "Dune: Part Two", "The Lord of the Rings: The Fellowship of the Ring", "The Lord of the Rings: The Two Towers"]'
        )]
)

In [60]:
vector_store_info.json()

'{"metadata_info": [{"name": "title", "type": "str", "description": "title of the movie, one of [\\"Dune (2021 film)\\", \\"Dune: Part Two\\", \\"The Lord of the Rings: The Fellowship of the Ring\\", \\"The Lord of the Rings: The Two Towers\\"]"}], "content_info": "semantic information about movies"}'

In [58]:
class AutoRetrieveModel(BaseModel):
    query: str = Field(..., description="natural language query string")
    filter_key_list: List[str] = Field(
        ..., description="List of metadata filter field names"
    )
    filter_value_list: List[str] = Field(
        ...,
        description=(
            "List of metadata filter field values (corresponding to names specified in filter_key_list)"
        )
    )

In [59]:
def auto_retrieve_fn(
    query: str, filter_key_list: List[str], filter_value_list: List[str]
):
    """Auto retrieval function.

    Performs auto-retrieval from a vector database, and then applies a set of filters.

    """
    query = query or "Query"

    exact_match_filters = [
        ExactMatchFilter(key=k, value=v)
        for k, v in zip(filter_key_list, filter_value_list)
    ]
    retriever = VectorIndexRetriever(
        index, filters=MetadataFilters(filters=exact_match_filters), top_k=top_k
    )
    query_engine = RetrieverQueryEngine.from_args(retriever)

    response = query_engine.query(query)
    return str(response)

In [61]:
description = f"""\
Use this tool to look up non-review based information about films.
The vector database schema is given below:
{vector_store_info.json()}
"""

auto_retrieve_tool = FunctionTool.from_defaults(
    fn=auto_retrieve_fn,
    name="semantic-film-info",
    description=description,
    fn_schema=AutoRetrieveModel
)

# ❓ Question #3:
Is the text in the description of our FunctionTool important or not? Please explain your answer.

In [62]:
from llama_index.core.agent import FunctionCallingAgentWorker

agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[auto_retrieve_tool],
    verbose=True,
)

agent = agent_worker.as_agent()

In [63]:
response = agent.chat("Who starred in the 2021 film?")


Added user message to memory: Who starred in the 2021 film?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: semantic-film-info with args: {"query": "Who starred in the film?", "filter_key_list": ["title"], "filter_value_list": ["Dune (2021 film)"]}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Function Output ===
The film starred Timothée Chalamet as Paul Atreides, Rebecca Ferguson as Lady Jessica, Oscar Isaac as Duke Leto, Zendaya as Chani, Dave Bautista as Glossu Rabban, Stellan Skarsgård as Baron Vladimir Harkonnen, Charlotte Rampling as

wandb: Logged trace tree to W&B.


=== LLM Response ===
The 2021 film "Dune" starred:

- Timothée Chalamet as Paul Atreides
- Rebecca Ferguson as Lady Jessica
- Oscar Isaac as Duke Leto
- Zendaya as Chani
- Dave Bautista as Glossu Rabban
- Stellan Skarsgård as Baron Vladimir Harkonnen
- Charlotte Rampling as Gaius Helen Mohiam
- Javier Bardem as Stilgar
- Josh Brolin as Gurney Halleck
- Jason Momoa as Duncan Idaho
- David Dastmalchian as Piter de Vries
- Stephen McKinley Henderson
- Chang Chen as Dr. Yueh
- Benjamin Clementine as the Herald of the Change


In [64]:
response = agent.chat("Who are the giant beings that roam across the world in the movies?")

Added user message to memory: Who are the giant beings that roam across the world in the movies?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: semantic-film-info with args: {"query": "giant beings that roam across the world", "filter_key_list": ["title"], "filter_value_list": ["Dune (2021 film)"]}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Function Output ===
The giant beings that roam across the world in the film are the sandworms. They travel under the desert and are a significant danger, especially during spice harvesting.
=== Calli

wandb: Logged trace tree to W&B.


=== LLM Response ===
In the movies:

- **Dune (2021 film)**: The giant beings that roam across the world are the **sandworms**. They travel under the desert and pose a significant danger, especially during spice harvesting.

- **The Lord of the Rings: The Fellowship of the Ring**: There is no specific mention of giant beings that roam across the world in this context.

- **The Lord of the Rings: The Two Towers**: **Treebeard** is an example of a giant being that roams across the world. He is an Ent, a tree-like creature, who interacts with Merry and Pippin in the film.


# Quantitative RAG Pipeline with NL2SQL Tooling

In [67]:
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune1.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune2.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/lotr_fotr.csv
!wget https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/lotr_tt.csv



--2024-07-09 22:10:35--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133391 (130K) [text/plain]
Saving to: ‘dune1.csv.1’

dune1.csv.1         100%[===================>] 130.26K   857KB/s    in 0.2s    

2024-07-09 22:10:35 (857 KB/s) - ‘dune1.csv.1’ saved [133391/133391]

--2024-07-09 22:10:36--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111843 (109K) [text/plain]
S

--2024-07-09 22:10:17--  https://raw.githubusercontent.com/AI-Maker-Space/DataRepository/main/dune2.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111843 (109K) [text/plain]
Saving to: ‘dune2.csv’

dune2.csv           100%[===================>] 109.22K  --.-KB/s    in 0.08s   

2024-07-09 22:10:18 (1.30 MB/s) - ‘dune2.csv’ saved [111843/111843]



### Read .csv Into Pandas

In [68]:
import pandas as pd

dune1 = pd.read_csv("./dune1.csv")
dune2 = pd.read_csv("./dune2.csv")
lotr_fotr = pd.read_csv("./lotr_fotr.csv")
lotr_tt = pd.read_csv("./lotr_tt.csv")

### Create SQLAlchemy engine with SQLite


In [69]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///:memory:")

### Convert pd.DataFrame to SQL tables


In [70]:
dune1.to_sql(
  "Dune (2021 film)",
  engine
)
dune2.to_sql(
  "Dune: Part Two",
  engine
)
lotr_fotr.to_sql(
  "The Lord of the Rings: The Fellowship of the Ring",
  engine
)
lotr_tt.to_sql(
  "The Lord of the Rings: The Two Towers",
  engine
)

149

### Construct a SQLDatabase index

In [71]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(
    engine=engine,
    include_tables=movie_list
)

### Create the NLSQLTableQueryEngine interface for all added SQL tables


In [72]:
from llama_index.core.indices.struct_store.sql_query import NLSQLTableQueryEngine

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=movie_list,
)

### Wrap It All Up in a QueryEngineTool


## Activity #1:

In [73]:
DESCRIPTION = """\
This tool should be used to answer any and all review related inquiries by translating a natural language query into a SQL query with access to tables:
'Dune (2021 film)' - containing info. about the first movie in the Dune series,
'Dune: Part Two'- containing info. about about the second movie in the Dune series,
'The Lord of the Rings: The Fellowship of the Ring' - containing info. about the first movie in the Lord of the Ring series,
'The Lord of the Rings: The Two Towers' - containing info. the second movie in the Lord of the Ring series
"""

In [74]:
from llama_index.core.tools.query_engine import QueryEngineTool

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql-query",
    description=DESCRIPTION,
)

In [75]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[sql_tool],
    verbose=True
)

agent = agent_worker.as_agent()

In [76]:
response = agent.chat("What is the average rating of the 2nd Lord of the Rings movie?")


Added user message to memory: What is the average rating of the 2nd Lord of the Rings movie?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: sql-query with args: {"input": "SELECT AVG(rating) as average_rating FROM 'The Lord of the Rings: The Two Towers'"}
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'Dune (2021 film)' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'Dune: Part Two' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'The Lord of the Rings: The Fellowship of the Ring' has columns: index (BIGINT), Unnamed: 0 (B

wandb: Logged trace tree to W&B.


=== LLM Response ===
The average rating of "The Lord of the Rings: The Two Towers" is approximately 9.18.


In [77]:
response = agent.chat("What movie series has better reviews, Lord of the Rings or Dune?")


Added user message to memory: What movie series has better reviews, Lord of the Rings or Dune?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: sql-query with args: {"input": "SELECT AVG(rating) as average_rating FROM 'The Lord of the Rings: The Fellowship of the Ring'"}
INFO:llama_index.core.indices.struct_store.sql_retriever:> Table desc str: Table 'Dune (2021 film)' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'Dune: Part Two' has columns: index (BIGINT), Unnamed: 0 (BIGINT), Review_Date (TEXT), Author (TEXT), Rating (FLOAT), Review_Title (TEXT), Review (TEXT), Review_Url (TEXT), and foreign keys: .

Table 'The Lord of the Rings: The Fellowship of the Ring' has columns: index (BIGINT),

wandb: Logged trace tree to W&B.


=== LLM Response ===
Here are the average ratings for each movie:

- **The Lord of the Rings: The Fellowship of the Ring**: 9.87
- **The Lord of the Rings: The Two Towers**: 9.18
- **Dune (2021)**: 8.34
- **Dune: Part Two**: 8.71

Based on these average ratings, the **Lord of the Rings** series has better reviews compared to the **Dune** series.


# Task 2: Combined RAG Pipeline


In [78]:
combined_tool_agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[auto_retrieve_tool, sql_tool],
    verbose=True
)

combined_tool_agent = combined_tool_agent_worker.as_agent()

In [79]:
response = combined_tool_agent.chat("Which movie is about a ring, and what is the average rating of the movie?")


Added user message to memory: Which movie is about a ring, and what is the average rating of the movie?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: semantic-film-info with args: {"query": "movie about a ring", "filter_key_list": ["title"], "filter_value_list": ["The Lord of the Rings: The Fellowship of the Ring", "The Lord of the Rings: The Two Towers"]}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Function Output ===
The movie is about the Dark Lord Sauron, who seeks the One Ring to return to power. The Ring has found its way to a you

wandb: Logged trace tree to W&B.


=== LLM Response ===
The movie about a ring is **"The Lord of the Rings: The Fellowship of the Ring"**. It tells the story of the Dark Lord Sauron, who seeks the One Ring to return to power. The Ring has found its way to a young hobbit named Frodo Baggins. Frodo and eight companions form the Fellowship of the Ring and embark on a perilous journey to Mount Doom in Mordor, the only place where the Ring can be destroyed.

The average rating for **"The Lord of the Rings: The Fellowship of the Ring"** is approximately **9.87**.


In [80]:
response = combined_tool_agent.chat("What worlds do the LoTR, and Dune movies take place in?")


Added user message to memory: What worlds do the LoTR, and Dune movies take place in?
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Calling Function ===
Calling function: semantic-film-info with args: {"query": "world where the movie takes place", "filter_key_list": ["title"], "filter_value_list": ["The Lord of the Rings: The Fellowship of the Ring", "The Lord of the Rings: The Two Towers"]}
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
=== Function Output ===
The movie takes place in the fictional world of Middle-earth.
=== Calling Function ===
Calling function: semantic-film-inf

wandb: Logged trace tree to W&B.


=== LLM Response ===
The worlds where the movies take place are as follows:

- **The Lord of the Rings: The Fellowship of the Ring** and **The Lord of the Rings: The Two Towers**: These movies take place in the fictional world of **Middle-earth**.

- **Dune (2021 film)** and **Dune: Part Two**: These movies take place in a distant future where humanity has spread across the universe and settled on various planets. The primary settings include the desert planet **Arrakis**, the oceanic planet **Caladan**, and the industrial planet **Giedi Prime**.


In [85]:
response = combined_tool_agent.chat("Which of the following movie series is considered the 'best': Dune, or Lord of the Rings? Base your answer on both reviews, and non-review information.")


Added user message to memory: Which of the following movie series is considered the 'best': Dune, or Lord of the Rings? Base your answer on both reviews, and non-review information.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


wandb: Logged trace tree to W&B.


=== LLM Response ===
### Reviews-Based Comparison

#### Average Ratings:
- **Dune (2021 film)**: 8.34
- **Dune: Part Two**: 8.71
- **The Lord of the Rings: The Fellowship of the Ring**: 9.87
- **The Lord of the Rings: The Two Towers**: 9.18

### Non-Review Information

#### Critical Acclaim and Awards:
- **Dune (2021 film)**:
  - Received generally positive critical acclaim.
  - Praise for ambition, story, scope, worldbuilding, performances, and production values.
  - Some criticism for being incomplete, dull, and mixed opinions on runtime, pacing, and adaptation.
  - Rotten Tomatoes: 83% positive reviews, average rating 7.6/10.
  - Metacritic: Weighted average score of 74/100.
  - Audience responses: CinemaScore average grade "A−", PostTrak 84% positive score.

- **The Lord of the Rings**:
  - Both "The Fellowship of the Ring" and "The Two Towers" are highly acclaimed.
  - Known for groundbreaking visual effects, storytelling, and faithful adaptation of J.R.R. Tolkien's work.
  - Mult

## ❓ Question #4:
How can you verify which tool was used for which query?

We can identify it from the logs. Logs has information like `=== Calling Function ===` which shows the details of the function called.

In [82]:
wandb_callback.finish()